In [9]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
from __future__ import division

In [117]:
# Load Original Data
df_train = pd.read_csv('../../data/beer/Train.csv')
df_valid = pd.read_csv('../../data/beer/Valid.csv')
df_test = pd.read_csv('../../data/beer/Test.csv')
key_phrase = pd.read_csv('../../data/beer/KeyPhrases.csv')

In [62]:
# Load U-I Data 
rtrain = load_npz("../../data/beer/Rtrain.npz")
# rvalid = load_npz("../../data/beer/Rvalid.npz")
# rtest = load_npz("../../data/beer/Rtest.npz")

In [6]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [8]:
df_train.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,Caldera- Pale Ale\t\tPicked up at Grape & Gour...,1251327677,NJpadreFan,0,1767,659,"['caldera', 'pale', 'ale', 'picked', 'grape', ...",caldera pale ale picked grape gourmet vb ligh...,"[1, 7, 12, 14, 16, 29, 31, 34, 39, 40, 42, 45]",12
1,5.5,Caldera Pale Ale,American Pale Ale (APA),2.5,12 oz. can poured into a pint glass. I really ...,1250928902,vacax,0,6159,659,"['oz', 'poured', 'pint', 'glass', 'really', 'l...",oz poured pint glass really like beer like pa...,"[1, 16, 18, 20, 24, 39, 42, 43, 45, 55]",10
2,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,A: Pours a cloudy orange with a crisp white tw...,1249866208,d0ggnate,0,3516,659,"['pours', 'cloudy', 'orange', 'crisp', 'white'...",pours cloudy orange crisp white two finger he...,"[12, 36, 39, 42]",4
3,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,Received this can through Beer Powerball. I am...,1249847121,babyhobbes,1,2893,659,"['received', 'beer', 'powerball', 'really', 's...",received beer powerball really sorry nt know ...,"[12, 42]",2
4,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,355ml can bought from Beersofeurope.com...\t\t...,1249556277,mdagnew,1,4858,659,"['bought', 'beersofeurope', 'com', 'poured', '...",bought beersofeurope com poured golden yellow...,"[1, 5, 7, 12, 14, 16, 18, 19, 26, 31, 34, 40, ...",18


In [1]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [36]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output

### Normalization

In [7]:
import numpy as np
x = np.random.rand(1000)*10
norm1 = x / np.linalg.norm(x)
norm2 = normalize(x[:,np.newaxis], axis=0).ravel()
print np.all(norm1 == norm2)
# True

True


# Explanation Prediction

Explanation could be a list of similar items or stated reasons in natural language. Keyphrases mined from user reviews are used to reflect reasons for user like or dislike an item. 

In [12]:
def explain(R,W2,k):
    """
    R: Rating Matrix with size U*I
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    Z: Joint Embedding/Latent Space with size U*U, generate r_ij and s_ij
    W2: Reconstruction matrix with size U*K 
    S: Output explanation prediction matrix with size U*K
    """
    Z = train(R)
    S = predict(W2, k, Z)
    return normalize(S)

In [13]:
Explanation_res1 = explain(rtrain, normalize(U_K), 100)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2103.70it/s]


In [21]:
np.argsort(Explanation_res1[0])[-20:]

array([18, 30, 11,  8, 58, 21, 45,  0, 50, 39,  6,  2, 42, 55, 12, 43, 44,
       16, 14,  1], dtype=int64)

In [20]:
np.argsort(np.ravel(normalize(U_K).todense()[0]))[-20:]

array([55, 26, 47, 62, 39, 12, 50,  6, 31, 58, 67, 21, 30, 44, 36, 16, 43,
        2,  1, 14], dtype=int64)

In [23]:
# Compare 
len(list(set(np.argsort(Explanation_res1[0])[-20:]).intersection(set(np.argsort(np.ravel(normalize(U_K).todense()[0]))[-20:]))))/20

0.7

In [33]:
def recall(vector_true_dense, vector_true_predict):
    """
    The fraction of relevant instances that have been retrieved over the total amount of relevant instances
    The length of vector_true_dense and vector_true_predict has to be the same
    Out put recall
    """
    hits = len(np.isin(vector_true_predict, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)

# Total Recall
def recall_all(true_matrix, predict_matrix, topK = 20):
    res = []
    for i in tqdm(range(len(Explanation_res1))):
        true_vector = np.argsort(np.ravel(normalize(true_matrix).todense()[i]))[-topK:]
        predict_vector = np.argsort(predict_matrix[i])[-topK:]
        res.append(recall(true_vector,predict_vector))
    return sum(res)/len(res)

In [34]:
recall_all(U_K,Explanation_res1)

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:30<00:00, 205.52it/s]


0.7153610675039237

In [39]:
Explanation_res2 = explain(rtrain, normalize(U_K), 200)
recall_all(U_K,Explanation_res2)

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:30<00:00, 206.12it/s]


0.7147723704866576

evaluate(Explanation_res1, normalize(U_K))

# Create a synthetic user u1

In [8]:
# U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))
U_K = load_npz('../../data/beer/U_K.npz')

In [70]:
def modify_user_keyphrase(U_K, keyphrase_ids, normalization = True, keyval = 1, user_id = 0):
    """
    Change all keyphrase id to some fixed number, all others to 0
    Return the U_K matrix with user_id row the synthetic user1
    """
    U_K[user_id,:] = 0
    for keyphrase_id in keyphrase_ids:
        U_K[user_id,keyphrase_id] = keyval
    if normalization == True:
        return normalize(U_K)
    return U_K

## Modify U_U latent Space 

but by changing/manipulating the U_I matrix, we need to know more information of each item. More specifically, we need to get a item-keyphrase matrix in order to change U_I as we want.

In [48]:
# I_K = get_I_K(df_train, row_name = 'ItemIndex', shape = (3668,75))
I_K = load_npz('../../data/beer/I_K.npz')

In [11]:
# I_K[:,0]: Items corresponding to Keyphrase 0
# Get top 100 items and reshape the matrix to array
keyphrase_id = 0
top_n = 100
np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:]

array([2972, 2212, 2244, 2169,  746, 2666, 3472,  946, 2034, 2698, 2451,
       2607, 2053, 2708, 1195,  160, 2178,  879, 1800, 1433, 2710, 2052,
       3046, 1021, 1490, 2255, 1690,  656, 3118,  764,  332, 3121,  448,
       1088, 1374, 3280, 2275,  305,  979, 2250,  875, 1898,  108, 2912,
       2928,  358,  751, 2125,  684, 3152, 2366, 2198, 2217, 3252,  331,
         41, 2978,  371, 3606,  363,  569,   19,  453, 1871, 1249, 1373,
       2744, 1323, 1202,  740, 2707, 2701, 2073,  184,  409, 1647,  268,
        128, 3143, 3147, 1290, 1198, 2209,  374, 2923, 2691, 3628, 2732,
       3631, 1371,  364, 2925,  361, 2874, 2468,  749, 3240, 2215, 1639,
        597], dtype=int64)

In [40]:
# Set some extreme U_I 
# For user1 set some items to 1
U_I[0,:] = 0
for i in np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:]:
    U_I[0,i] = 1
# U_I[0][np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:]] = 1

C:\Users\hanze\Anaconda3\envs\KNN\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [45]:
def item_associated_with_keyphrase(I_K, keyphrase_ids, top_n = 100):
    """
    I_K: Item Keyphrase Matrix
    Keyphrase_ids: top items described by keyphrase_ids
    output item list (list of list)
    """
    res = []
    for keyphrase_id in keyphrase_ids:
        res.append(np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:])
    return np.unique(res)

In [115]:
item_associated_with_keyphrase(I_K, [0])

array([  19,   41,  108,  128,  160,  184,  268,  305,  331,  332,  358,
        361,  363,  364,  371,  374,  409,  448,  453,  569,  597,  656,
        684,  740,  746,  749,  751,  764,  875,  879,  946,  979, 1021,
       1088, 1195, 1198, 1202, 1249, 1290, 1323, 1371, 1373, 1374, 1433,
       1490, 1639, 1647, 1690, 1800, 1871, 1898, 2034, 2052, 2053, 2073,
       2125, 2169, 2178, 2198, 2209, 2212, 2215, 2217, 2244, 2250, 2255,
       2275, 2366, 2451, 2468, 2607, 2666, 2691, 2698, 2701, 2707, 2708,
       2710, 2732, 2744, 2874, 2912, 2923, 2925, 2928, 2972, 2978, 3046,
       3118, 3121, 3143, 3147, 3152, 3240, 3252, 3280, 3472, 3606, 3628,
       3631], dtype=int64)

In [262]:
# Get the index corresponding to item id
# item_idx = 3644
# df_train.index[df_train['ItemIndex'] == item_idx].tolist()

In [185]:
def modify_user_preference(U_I, items, user_id = 0):
    """
    TODO: Fix the function s.t. it will not modify the initial matrix
    """
    U_I[user_id,:] = 0
    for i in items:
        U_I[0,i] = 1
    return U_I

### Experiment

In [238]:
U_K = load_npz('../../data/beer/U_K.npz')
U_K = normalize(U_K)
rtrain = load_npz("../../data/beer/Rtrain.npz")

In [204]:
items = item_associated_with_keyphrase(I_K, [8]) # coffee
U_I = modify_user_preference(rtrain, items)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:5: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  """


In [205]:
modified_U_U = train(U_I)
modified_U_K = modify_user_keyphrase(U_K,keyphrase_ids=[8])

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:6: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  


In [206]:
k = 100
user_1_keyphrase = normalize(predict(modified_U_K, k, modified_U_U))[0]
print user_1_keyphrase

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2024.79it/s]


[0.2106391  0.41475516 0.20996175 0.02593623 0.0680651  0.08887984
 0.2234473  0.02981935 0.17394346 0.04713533 0.08481757 0.0882533
 0.17009241 0.04244185 0.28178836 0.03119407 0.30238048 0.04481298
 0.08553967 0.04253654 0.05203158 0.15991004 0.06030151 0.04327027
 0.07178276 0.0108858  0.05164822 0.02825977 0.03723946 0.02047524
 0.05489275 0.06511377 0.02588423 0.00900447 0.06137628 0.02266214
 0.08221878 0.00615714 0.04238288 0.13852531 0.05251514 0.09014796
 0.17720271 0.24434653 0.27032971 0.1327618  0.01477079 0.08005493
 0.02965783 0.00767421 0.20038679 0.06749889 0.03210295 0.02347733
 0.0182822  0.22481009 0.04244211 0.01150589 0.11850405 0.02153989
 0.04717489 0.02386378 0.04588161 0.02651843 0.03030469 0.0319958
 0.02474507 0.05071973 0.05128094 0.01421376 0.01117445 0.032857
 0.01658942 0.01116816 0.03050019]


In [207]:
user1_top_keyphrase = np.argsort(user_1_keyphrase)[-20:]

In [239]:
def item_associated_with_keyphrase(I_K, keyphrase_ids, top_items = 100):
    """
    I_K: Item Keyphrase Matrix
    Keyphrase_ids: top items described by keyphrase_ids
    top_n: how many top items described by the keyphrases will be output 
    output item list (unique)
    """
    res = []
    for keyphrase_id in keyphrase_ids:
        res.append(np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_items:])
    return np.unique(res)

def modify_user_preference(U_I, items, user_id = 0):
    """
    TODO: Fix the function s.t. it will not modify the initial matrix
    """
    U_I[user_id,:] = 0
    for i in items:
        U_I[0,i] = 1
    return U_I

def explain_synthetic_user(rtrain, U_K, I_K, keyphrases, top_keyphrase = 20, user_id = 0, k = 100, top_items = 100):
    items = item_associated_with_keyphrase(I_K, keyphrases, top_items = top_items) # 8 is coffee
    U_I = modify_user_preference(rtrain, items, user_id=user_id)
    modified_U_U = train(U_I)
    U_K[user_id, :] = 0
    synthetic_user_keyphrase = normalize(predict(U_K, k, modified_U_U))[user_id]
    return np.argsort(synthetic_user_keyphrase)[-top_keyphrase:]

In [240]:
explain_synthetic_user(rtrain, U_K, I_K, [0])

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:17: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:26: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 1983.81it/s]


array([36, 11,  5, 58, 45, 39, 12, 21,  8, 42, 50,  0,  2,  6, 55, 43, 44,
       14, 16,  1], dtype=int64)

In [244]:
explain_synthetic_user(rtrain, U_K, I_K, [12])

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2016.46it/s]


array([50, 47, 31, 58, 18, 36,  0,  6, 21, 45, 39, 42,  2, 44, 55, 43, 12,
       14, 16,  1], dtype=int64)

In [261]:
keyphrase[12]

'ale'

In [258]:
# Get the average rank 
res1 = 0
for i in range(75):
    rtrain = load_npz("../../data/beer/Rtrain.npz")
    a = explain_synthetic_user(rtrain, U_K, I_K, [i],top_keyphrase=75)
    res1 += list(a[::-1]).index(i)
    print list(a[::-1]).index(i)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:17: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2090.58it/s]


8


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2096.77it/s]


0


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2005.67it/s]


...

59


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2089.21it/s]


46


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2091.27it/s]


30


In [260]:
# Average rank 
res1/75

28.026666666666667

# Evaluate the result

Since we create a sterotype user1 who only likes keyphrase 0, and let him liked some beer with keyphrase 0, we can evaluate the result U_K (user1's updated keyphrase preferance) by looking into what's the top K keyphrase he likes other than keyphrase 0. Then we look into: among the top N (100 in this case) beer with keyphrase 0, whats the next K (e.g. 10) most frequent keyphrase that appear, and compare this with the updated_U_K to see if user1 actually likes the other K keyphrase as we predicted. 

In [21]:
# For each item, whats the keyphrase (get the keyphrase collection)
def  get_keyphrases(item_list):
    """
    item_list: list of integer index of items
    output: list of items' keyphrase
    """
    keyphrases = []
    for item in item_list:
        keyphrase = np.ravel(I_K[item,:].todense()) # Change to np array and reshape
        keyphrases.append(keyphrase)
    return keyphrases

# Critiquing Draft

##### TF-IDF to get U_K

## Get the keyphrases from dataframe

In [221]:
def get_keyphrase(df_train):
    keyphrase = [0]*75
    check_flg = 0
    for i in range(len(df_train['keyVector'])):
        keyVector = ast.literal_eval(df_train['keyVector'][i])
        
        for j in range(len(keyVector)):
            if keyphrase[keyVector[j]] == 0:
                keyphrase[keyVector[j]] = ast.literal_eval(df_train['review'][i])[j]
                check_flg += 1
#                 print keyphrase
                if check_flg == 75:
                    return keyphrase
    return keyphrase

In [222]:
keyphrase = get_keyphrase(df_train)

In [223]:
keyphrase

['poured',
 'caldera',
 'poured',
 'pint',
 'clear',
 'beersofeurope',
 'pale',
 'pale',
 'black',
 'enjoying',
 'received',
 'pours',
 'ale',
 'way',
 'picked',
 'color',
 'grape',
 'perfect',
 'pint',
 'slight',
 'glass',
 'color',
 'coming',
 'mahogany',
 'really',
 'head',
 'haze',
 'dark',
 'glass',
 'gourmet',
 'golden',
 'vb',
 'brew',
 'foamy',
 'light',
 'football',
 'cloudy',
 'small',
 'well',
 'golden',
 'orange',
 'creamy',
 'small',
 'like',
 'hoppy',
 'fluffy',
 'beige',
 'review',
 'jon',
 'pours',
 'new',
 'foamy',
 'good',
 'color',
 'head',
 'ale',
 'hell',
 'back',
 'stick',
 'even',
 'source',
 'pretty',
 'thin',
 'gold',
 'oxidation',
 'hoppy',
 'expect',
 'crown',
 'decent',
 'better',
 'mango',
 'color',
 'colored',
 'leaf',
 'light']